# Data Cleaning (MC2)

Dataset Description (from VAST 2024 Docs):
- **Graph Type**: Directed Multi-Graph (multiple edges between nodes)
- **Nodes**: 5637
- **Edges**: 271752
- **Connected Component**: 1 (weakly connected)
- **Format**: JSON export from NetworkX (`node_link_data`)
- **Loading**: Use `node_link_graph()` from NetworkX

The scope of this notebook is to understand the structure and metadata of the provided directed multi-graph:
- Types of nodes and edges
- Volume of data (nodes, edges)
- Metadata and relevant attributes
- Prepare for visual analytics and filtering

### Load the Data

The `mc2.json` file is too large to include in the Git repository and is therefore listed in `.gitignore`.

To obtain the original dataset, download it from the official [VAST Challenge 2024 – Mini Challenge 2 (MC2) page](https://vast-challenge.github.io/2024/MC2.html).

Once downloaded, place the file in the `data/` directory (or your preferred location).  
Make sure your code points to the correct file path when loading the data.

In [1]:
import json
import pandas as pd
import networkx as nx

with open("mc2.json", "r") as f:
    data = json.load(f)

# Create a NetworkX multigraph
G = nx.node_link_graph(data, directed=True, multigraph=True)

print(f"Graph loaded. Total nodes: {G.number_of_nodes()}, total edges: {G.number_of_edges()}")

Graph loaded. Total nodes: 5637, total edges: 271643


## Nodes

In [2]:
# Extract node attributes into a DataFrame
node_data = pd.DataFrame(dict(G.nodes(data=True))).T

node_data.columns

Index(['type', '_last_edited_by', '_date_added', '_last_edited_date',
       '_raw_source', '_algorithm', 'name', 'Name', 'Description',
       'Activities', 'kind', 'qty_tons', 'date', 'flag_country', 'company',
       'tonnage', 'length_overall', 'style', 'fish_species_present'],
      dtype='object')

In [3]:
metadata = ['_last_edited_by', '_last_edited_date', '_date_added', '_raw_source', '_algorithm']

for col in metadata:
    print(f"{col}: {len(node_data[col].unique())} unique values")

_last_edited_by: 11 unique values
_last_edited_date: 500 unique values
_date_added: 525 unique values
_raw_source: 4 unique values
_algorithm: 3 unique values


In [4]:
print(node_data['_last_edited_by'].value_counts(), '\n')

print(node_data['_raw_source'].value_counts(), '\n')

print(node_data['_algorithm'].value_counts(), '\n')


_last_edited_by
Jack Inch           1145
Junior Shurdlu      1116
Melinda Manning     1112
Olokun Daramola     1110
Harvey Janus        1107
Kristin Baker         11
Greta Grass-Hill      10
juniorshurdlu794      10
Clepper Jessen         7
Haenyeo Hyun-Ki        6
Urashima Tarō          3
Name: count, dtype: int64 

_raw_source
Tuna Shelf/egress report         5307
Oceanus Vessel Locator System     296
Oceanus: Geography Notes           24
                                   10
Name: count, dtype: int64 

_algorithm
CatchMate ('arrrr' edition)    5307
OVLS-Catch&Hook                 296
                                 34
Name: count, dtype: int64 



In [5]:
# print range of dates '_date_added' and '_last_edited_date'
print("Date range for '_date_added':", node_data['_date_added'].min(), "to", node_data['_date_added'].max())
print("Date range for '_last_edited_date':", node_data['_last_edited_date'].min(), "to", node_data['_last_edited_date'].max())

Date range for '_date_added': 2033-06-14 to 2035-12-02
Date range for '_last_edited_date': 2033-07-18 to 2035-12-04


Since the metadata looks useless, we drop them:

In [6]:
# Drop metadata columns
node_data = node_data.drop(columns=metadata)

Now, let's analyze each node type...

In [7]:
# Count node types
print("NODE TYPES:")
print(node_data['type'].value_counts())

NODE TYPES:
type
Entity.Document.DeliveryReport    5307
Entity.Vessel.FishingVessel        178
Entity.Vessel.CargoVessel          100
Entity.Location.Point               12
Entity.Commodity.Fish               10
Entity.Location.City                 6
Entity.Vessel.Tour                   6
Entity.Location.Region               6
Entity.Vessel.Other                  5
Entity.Vessel.Ferry.Passenger        3
Entity.Vessel.Ferry.Cargo            2
Entity.Vessel.Research               2
Name: count, dtype: int64


In [8]:
# Display a few nodes of each type
node_data.groupby('type').head(1)

,type,name,Name,Description,Activities,kind,qty_tons,date,flag_country,company,tonnage,length_overall,style,fish_species_present
gadusnspecificatae4ba,Entity.Commodity.Fish,Cod/Gadus n.specificatae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City of Haacklee,Entity.Location.City,NaN,Haacklee,None,"[Tourism, Local shipping]",city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cargo_2035_2394778c,Entity.Document.DeliveryReport,NaN,NaN,NaN,NaN,NaN,24.375,2035-11-03,NaN,NaN,NaN,NaN,NaN,NaN
wavewranglerc2d,Entity.Vessel.FishingVessel,NaN,Wave Wrangler,NaN,NaN,NaN,NaN,NaN,Oceanus,"Roth, Logan and Moreno",700,110,NaN,NaN
webigailba7,Entity.Vessel.Other,NaN,Webigail,NaN,NaN,NaN,NaN,NaN,Oceanus,NaN,NaN,50,NaN,NaN
venerable89c,Entity.Vessel.Ferry.Passenger,NaN,Venerable,NaN,NaN,NaN,NaN,NaN,Oceanus,NaN,NaN,NaN,NaN,NaN
seawaysavvy9847,Entity.Vessel.CargoVessel,NaN,Seaway Savvy 9,NaN,NaN,NaN,NaN,NaN,Osterivaro,NaN,2100,90,NaN,NaN
sturdyd7f,Entity.Vessel.Ferry.Cargo,NaN,Sturdy,NaN,NaN,NaN,NaN,NaN,Oceanus,NaN,NaN,NaN,NaN,NaN
inquisitive8c0,Entity.Vessel.Research,NaN,Inquisitive,NaN,NaN,NaN,NaN,NaN,Oceanus,NaN,NaN,NaN,NaN,NaN
himarkroyal032,Entity.Vessel.Tour,NaN,Himark Royal,NaN,NaN,NaN,NaN,NaN,Oceanus,NaN,NaN,NaN,NaN,NaN


In [9]:
# FUNCTION to delete all columns where all values are NaN
def drop_na_cols(df):
    all_nan_cols = df.columns[df.isna().all()].tolist()
    df = df.drop(columns=all_nan_cols)
    print("Dropped columns (all NaN):", all_nan_cols)
    return df

In [10]:
node_data = drop_na_cols(node_data)

Dropped columns (all NaN): []


### Entity.Location

In [11]:
points = cities = node_data[node_data['type'] == 'Entity.Location.Point']
cities = node_data[node_data['type'] == 'Entity.Location.City']
regions = node_data[node_data['type'] == 'Entity.Location.Region']

In [12]:
points

,type,name,Name,Description,Activities,kind,qty_tons,date,flag_country,company,tonnage,length_overall,style,fish_species_present
Exit West,Entity.Location.Point,NaN,Exit West,None,"[International shipping, Deep sea fishing]",buoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nav 3,Entity.Location.Point,NaN,Nav 3,Navigation in/out of South Paackland,[Navigation],buoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nav D,Entity.Location.Point,NaN,Nav D,Safe navigation around Nemo Reef,[Navigation],buoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nav B,Entity.Location.Point,NaN,Nav B,Safe Navigation around Limpet Preserve,[Navigation],buoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nav A,Entity.Location.Point,NaN,Nav A,Safe Navigation around Limpet Preserve,[Navigation],buoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nav C,Entity.Location.Point,NaN,Nav C,Safe Navigation near Lomark,[Navigation],buoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nav 2,Entity.Location.Point,NaN,Nav 2,Safe navigation near Ghoti Preserve and Paackland,[Navigation],buoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nav 1,Entity.Location.Point,NaN,Nav 1,Navigation north of Ghoti Preserve,[Navigation],buoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Exit East,Entity.Location.Point,NaN,Exit East,None,"[International shipping, Deep sea fishing]",buoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Exit South,Entity.Location.Point,NaN,Exit South,None,"[International shipping, Deep sea fishing]",buoy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
cities

,type,name,Name,Description,Activities,kind,qty_tons,date,flag_country,company,tonnage,length_overall,style,fish_species_present
City of Haacklee,Entity.Location.City,NaN,Haacklee,None,"[Tourism, Local shipping]",city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City of Lomark,Entity.Location.City,NaN,Lomark,None,"[Deep sea fishing, Commercial fishing, Tourism...",city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City of Himark,Entity.Location.City,NaN,Himark,None,"[Recreation, tourism]",city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City of Paackland,Entity.Location.City,NaN,Paackland,None,"[Industry, Fishing industry, Local shipping, T...",city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City of South Paackland,Entity.Location.City,NaN,South Paackland,None,"[Industry, Fishing industry, Local shipping]",city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City of Port Grove,Entity.Location.City,NaN,Port Grove,None,"[Tourism, Research]",city,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
regions

,type,name,Name,Description,Activities,kind,qty_tons,date,flag_country,company,tonnage,length_overall,style,fish_species_present
Cod Table,Entity.Location.Region,NaN,Cod Table,,[Commercial fishing],Fishing Ground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Cod/Gadus n.specificatae, Birdseye/Pisces fri..."
Ghoti Preserve,Entity.Location.Region,NaN,Ghoti Preserve,,"[Research, Tourism, Recreation]",Ecological Preserve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Wrasse/Labridae n.refert, Beauvoir/Habeas pis..."
Wrasse Beds,Entity.Location.Region,NaN,Wrasse Beds,,[Commercial fishing],Fishing Ground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Wrasse/Labridae n.refert, Birdseye/Pisces fri..."
Nemo Reef,Entity.Location.Region,NaN,Nemo Reef,,"[Recreation, Tourism]",Ecological Preserve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Wrasse/Labridae n.refert, Tuna/Thunnini n.ver..."
Don Limpet Preserve,Entity.Location.Region,NaN,Don Limpet Preserve,,"[Recreation, Tourism]",Ecological Preserve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tuna/Thunnini n.vera, Birdseye/Pisces frigus,..."
Tuna Shelf,Entity.Location.Region,NaN,Tuna Shelf,,"[Commercial fishing, Sport fishing]",Fishing Ground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Tuna/Thunnini n.vera, Birdseye/Pisces frigus,..."


We now have the illegal regions for fishing:

In [15]:
illegal_fishing_locations = ['Ghoti Preserve', 'Nemo Reef', 'Don Limpet Preserve']

In [16]:
location_metadata = ['Name', 'Activities', 'Description', 'fish_species_present', 'kind']

# Print missing values for each location metadata field
print('REGIONS:')
for meta in location_metadata:
    missing = regions[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(regions)} total points")

print('\nCITIES:')
for meta in location_metadata:
    missing = cities[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(cities)} total points")

print('\nPOINTS:')
for meta in location_metadata:
    missing = points[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(points)} total points")

REGIONS:
Missing values in 'Name': 0 out of 6 total points
Missing values in 'Activities': 0 out of 6 total points
Missing values in 'Description': 0 out of 6 total points
Missing values in 'fish_species_present': 0 out of 6 total points
Missing values in 'kind': 0 out of 6 total points

CITIES:
Missing values in 'Name': 0 out of 6 total points
Missing values in 'Activities': 0 out of 6 total points
Missing values in 'Description': 6 out of 6 total points
Missing values in 'fish_species_present': 6 out of 6 total points
Missing values in 'kind': 0 out of 6 total points

POINTS:
Missing values in 'Name': 0 out of 12 total points
Missing values in 'Activities': 0 out of 12 total points
Missing values in 'Description': 4 out of 12 total points
Missing values in 'fish_species_present': 12 out of 12 total points
Missing values in 'kind': 0 out of 12 total points


In [17]:
points = drop_na_cols(points)
cities = drop_na_cols(cities)
regions = drop_na_cols(regions)

Dropped columns (all NaN): ['name', 'qty_tons', 'date', 'flag_country', 'company', 'tonnage', 'length_overall', 'style', 'fish_species_present']
Dropped columns (all NaN): ['name', 'Description', 'qty_tons', 'date', 'flag_country', 'company', 'tonnage', 'length_overall', 'style', 'fish_species_present']
Dropped columns (all NaN): ['name', 'qty_tons', 'date', 'flag_country', 'company', 'tonnage', 'length_overall', 'style']


In [18]:
points

,type,Name,Description,Activities,kind
Exit West,Entity.Location.Point,Exit West,None,"[International shipping, Deep sea fishing]",buoy
Nav 3,Entity.Location.Point,Nav 3,Navigation in/out of South Paackland,[Navigation],buoy
Nav D,Entity.Location.Point,Nav D,Safe navigation around Nemo Reef,[Navigation],buoy
Nav B,Entity.Location.Point,Nav B,Safe Navigation around Limpet Preserve,[Navigation],buoy
Nav A,Entity.Location.Point,Nav A,Safe Navigation around Limpet Preserve,[Navigation],buoy
Nav C,Entity.Location.Point,Nav C,Safe Navigation near Lomark,[Navigation],buoy
Nav 2,Entity.Location.Point,Nav 2,Safe navigation near Ghoti Preserve and Paackland,[Navigation],buoy
Nav 1,Entity.Location.Point,Nav 1,Navigation north of Ghoti Preserve,[Navigation],buoy
Exit East,Entity.Location.Point,Exit East,None,"[International shipping, Deep sea fishing]",buoy
Exit South,Entity.Location.Point,Exit South,None,"[International shipping, Deep sea fishing]",buoy


### Entity.Commodity.Fish

In [19]:
commodities = node_data[node_data['type'] == 'Entity.Commodity.Fish']

commodities

,type,name,Name,Description,Activities,kind,qty_tons,date,flag_country,company,tonnage,length_overall,style,fish_species_present
gadusnspecificatae4ba,Entity.Commodity.Fish,Cod/Gadus n.specificatae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
piscesfrigus900,Entity.Commodity.Fish,Birdseye/Pisces frigus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
piscesfoetidaae7,Entity.Commodity.Fish,Sockfish/Pisces foetida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
labridaenrefert9be,Entity.Commodity.Fish,Wrasse/Labridae n.refert,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
habeaspisces4eb,Entity.Commodity.Fish,Beauvoir/Habeas pisces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
piscissapidum9b7,Entity.Commodity.Fish,Harland/Piscis sapidum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
thunnininveradb7,Entity.Commodity.Fish,Tuna/Thunnini n.vera,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
piscisosseusb6d,Entity.Commodity.Fish,Offidiaa/Piscis osseus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
oncorhynchusrosea790,Entity.Commodity.Fish,Salmon/Oncorhynchus rosea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
piscessatisb87,Entity.Commodity.Fish,Helenaa/Pisces satis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
commodity_metadata = ['name']

# Print missing values for each commodity metadata field
for meta in commodity_metadata:
    missing = commodities[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(commodities)} total points")

Missing values in 'name': 0 out of 10 total points


In [21]:
commodities = drop_na_cols(commodities)

Dropped columns (all NaN): ['Name', 'Description', 'Activities', 'kind', 'qty_tons', 'date', 'flag_country', 'company', 'tonnage', 'length_overall', 'style', 'fish_species_present']


### Entity.Vessel

In [22]:
vessel_types = ['Entity.Vessel.FishingVessel', 'Entity.Vessel.Ferry.Passenger', 'Entity.Vessel.CargoVessel', 'Entity.Vessel.Ferry.Cargo', 'Entity.Vessel.Tour', 'Entity.Vessel.Research', 'Entity.Vessel.Other']
vessels = node_data[node_data['type'].isin(vessel_types)]

fishing_vessels = vessels[vessels['type'] == 'Entity.Vessel.FishingVessel']
ferry_vessels = vessels[vessels['type'] == 'Entity.Vessel.Ferry.Passenger']
cargo_vessels = vessels[vessels['type'] == 'Entity.Vessel.CargoVessel']
ferry_cargo_vessels = vessels[vessels['type'] == 'Entity.Vessel.Ferry.Cargo']
tour_vessels = vessels[vessels['type'] == 'Entity.Vessel.Tour']
research_vessels = vessels[vessels['type'] == 'Entity.Vessel.Research']
other_vessels = vessels[vessels['type'] == 'Entity.Vessel.Other']

vessels.head()

,type,name,Name,Description,Activities,kind,qty_tons,date,flag_country,company,tonnage,length_overall,style,fish_species_present
wavewranglerc2d,Entity.Vessel.FishingVessel,NaN,Wave Wrangler,NaN,NaN,NaN,NaN,NaN,Oceanus,"Roth, Logan and Moreno",700,110,NaN,NaN
yellowfintunataker08b,Entity.Vessel.FishingVessel,NaN,Yellowfin Tuna Taker,NaN,NaN,NaN,NaN,NaN,Oceanus,Brown-Haas,4500,110,NaN,NaN
webigailba7,Entity.Vessel.Other,NaN,Webigail,NaN,NaN,NaN,NaN,NaN,Oceanus,NaN,NaN,50,NaN,NaN
arcticgraylingangler094,Entity.Vessel.FishingVessel,NaN,Arctic Grayling Angler,NaN,NaN,NaN,NaN,NaN,Oceanus,"Smith, Davis and Acosta",1500,80,NaN,NaN
anchovyassaulterb1c,Entity.Vessel.FishingVessel,NaN,Anchovy Assaulter,NaN,NaN,NaN,NaN,NaN,Oceanus,Cisneros-Meyer,400,60,NaN,NaN


In [23]:
vessel_metadata = ['Name', 'company', 'flag_country', 'length_overall', 'tonnage']

# Print missing values for each vessel metadata field
print('FISHING VESSELS:')
for meta in vessel_metadata:
    missing = fishing_vessels[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(fishing_vessels)} total points")

print('\nFERRY VESSELS:')
for meta in vessel_metadata:
    missing = ferry_vessels[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(ferry_vessels)} total points")

print('\nCARGO VESSELS:')
for meta in vessel_metadata:
    missing = cargo_vessels[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(cargo_vessels)} total points")

print('\nFERRY CARGO VESSELS:')
for meta in vessel_metadata:
    missing = ferry_cargo_vessels[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(ferry_cargo_vessels)} total points")

print('\nTOUR VESSELS:')
for meta in vessel_metadata:
    missing = tour_vessels[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(tour_vessels)} total points")

print('\nRESEARCH VESSELS:')
for meta in vessel_metadata:
    missing = research_vessels[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(research_vessels)} total points")

print('\nOTHER VESSELS:')
for meta in vessel_metadata:
    missing = other_vessels[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(other_vessels)} total points")

FISHING VESSELS:
Missing values in 'Name': 0 out of 178 total points
Missing values in 'company': 0 out of 178 total points
Missing values in 'flag_country': 0 out of 178 total points
Missing values in 'length_overall': 0 out of 178 total points
Missing values in 'tonnage': 0 out of 178 total points

FERRY VESSELS:
Missing values in 'Name': 0 out of 3 total points
Missing values in 'company': 3 out of 3 total points
Missing values in 'flag_country': 0 out of 3 total points
Missing values in 'length_overall': 3 out of 3 total points
Missing values in 'tonnage': 3 out of 3 total points

CARGO VESSELS:
Missing values in 'Name': 0 out of 100 total points
Missing values in 'company': 99 out of 100 total points
Missing values in 'flag_country': 0 out of 100 total points
Missing values in 'length_overall': 0 out of 100 total points
Missing values in 'tonnage': 0 out of 100 total points

FERRY CARGO VESSELS:
Missing values in 'Name': 0 out of 2 total points
Missing values in 'company': 2 out o

In [24]:
fishing_vessels = drop_na_cols(fishing_vessels)
ferry_vessels = drop_na_cols(ferry_vessels)
cargo_vessels = drop_na_cols(cargo_vessels)
ferry_cargo_vessels = drop_na_cols(ferry_cargo_vessels)
tour_vessels = drop_na_cols(tour_vessels)
research_vessels = drop_na_cols(research_vessels)
other_vessels = drop_na_cols(other_vessels)

Dropped columns (all NaN): ['name', 'Description', 'Activities', 'kind', 'qty_tons', 'date', 'style', 'fish_species_present']
Dropped columns (all NaN): ['name', 'Description', 'Activities', 'kind', 'qty_tons', 'date', 'company', 'tonnage', 'length_overall', 'style', 'fish_species_present']
Dropped columns (all NaN): ['name', 'Description', 'Activities', 'kind', 'qty_tons', 'date', 'style', 'fish_species_present']
Dropped columns (all NaN): ['name', 'Description', 'Activities', 'kind', 'qty_tons', 'date', 'company', 'tonnage', 'length_overall', 'style', 'fish_species_present']
Dropped columns (all NaN): ['name', 'Description', 'Activities', 'kind', 'qty_tons', 'date', 'company', 'tonnage', 'length_overall', 'fish_species_present']
Dropped columns (all NaN): ['name', 'Description', 'Activities', 'kind', 'qty_tons', 'date', 'company', 'tonnage', 'length_overall', 'style', 'fish_species_present']
Dropped columns (all NaN): ['name', 'Description', 'Activities', 'kind', 'qty_tons', 'date', 

### Entity.Document.DeliveryReport

In [25]:
documents = node_data[node_data['type'] == 'Entity.Document.DeliveryReport']

documents.head()

,type,name,Name,Description,Activities,kind,qty_tons,date,flag_country,company,tonnage,length_overall,style,fish_species_present
cargo_2035_2394778c,Entity.Document.DeliveryReport,NaN,NaN,NaN,NaN,NaN,24.375,2035-11-03,NaN,NaN,NaN,NaN,NaN,NaN
cargo_2035_23956ba0,Entity.Document.DeliveryReport,NaN,NaN,NaN,NaN,NaN,18.125,2035-08-16,NaN,NaN,NaN,NaN,NaN,NaN
cargo_2035_23957cfd,Entity.Document.DeliveryReport,NaN,NaN,NaN,NaN,NaN,20.625,2035-08-20,NaN,NaN,NaN,NaN,NaN,NaN
cargo_2035_23958501,Entity.Document.DeliveryReport,NaN,NaN,NaN,NaN,NaN,13.125,2035-11-07,NaN,NaN,NaN,NaN,NaN,NaN
cargo_2035_23959ab6,Entity.Document.DeliveryReport,NaN,NaN,NaN,NaN,NaN,13.125,2035-08-24,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
document_metadata = ['qty_tons', 'date']

for meta in document_metadata:
    missing = documents[meta].isnull().sum()
    print(f"Missing values in '{meta}': {missing} out of {len(documents)} total points")

Missing values in 'qty_tons': 0 out of 5307 total points
Missing values in 'date': 0 out of 5307 total points


In [27]:
documents = drop_na_cols(documents)

Dropped columns (all NaN): ['name', 'Name', 'Description', 'Activities', 'kind', 'flag_country', 'company', 'tonnage', 'length_overall', 'style', 'fish_species_present']


### *Overview table* with note types, instance count and retained columns for each

In [28]:
summary_list = []

dfs = [
    ('documents', documents),
    ('fishing_vessels', fishing_vessels),
    ('cargo_vessels', cargo_vessels),
    ('ferry_vessels', ferry_vessels),
    ('ferry_cargo_vessels', ferry_cargo_vessels),
    ('tour_vessels', tour_vessels),
    ('research_vessels', research_vessels),
    ('other_vessels', other_vessels),
    ('commodities', commodities),
    ('points', points),
    ('cities', cities),
    ('regions', regions),
]

for name, df in dfs:
    note_type = df['type'].iloc[0] if not df.empty and 'type' in df.columns else 'N/A'
    summary_list.append({
        'DataFrame': name,
        'Node Type': note_type,
        'Count': len(df),
        'Key Columns': ', '.join(df.columns)
    })

summary_df = pd.DataFrame(summary_list)
print(summary_df.to_string(index=False))

          DataFrame                      Node Type  Count                                                     Key Columns
          documents Entity.Document.DeliveryReport   5307                                            type, qty_tons, date
    fishing_vessels    Entity.Vessel.FishingVessel    178      type, Name, flag_country, company, tonnage, length_overall
      cargo_vessels      Entity.Vessel.CargoVessel    100      type, Name, flag_country, company, tonnage, length_overall
      ferry_vessels  Entity.Vessel.Ferry.Passenger      3                                        type, Name, flag_country
ferry_cargo_vessels      Entity.Vessel.Ferry.Cargo      2                                        type, Name, flag_country
       tour_vessels             Entity.Vessel.Tour      6                                 type, Name, flag_country, style
   research_vessels         Entity.Vessel.Research      2                                        type, Name, flag_country
      other_vessels     

## Edges

In [29]:
# Extract all edges with their attributes into a clean list of dictionaries
edge_data = pd.DataFrame([
    {
        "source": u,
        "target": v,
        "key": k,
        **attr
    }
    for u, v, k, attr in G.edges(data=True, keys=True)
])

edge_data.columns.tolist()

['source',
 'target',
 'key',
 'type',
 'time',
 'dwell',
 '_last_edited_by',
 '_date_added',
 '_last_edited_date',
 '_raw_source',
 '_algorithm',
 'date',
 'data_author',
 'aphorism',
 'holiday_greeting',
 'wisdom',
 'saying of the sea']

In [30]:
metadata = ['_last_edited_by', '_last_edited_date', '_date_added', '_raw_source', '_algorithm']

for col in metadata:
    print(f"{col}: {len(edge_data[col].unique())} unique values")

_last_edited_by: 8 unique values
_last_edited_date: 32521 unique values
_date_added: 17308 unique values
_raw_source: 8 unique values
_algorithm: 3 unique values


Again, we drop the metadata columns:

In [31]:
# Drop metadata columns
edge_data = edge_data.drop(columns=metadata)

We analyze the edge types one by one...

In [32]:
# Count edge types
print("EDGE TYPES:")
print(edge_data['type'].value_counts())

EDGE TYPES:
type
Event.TransportEvent.TransponderPing    258542
Event.Transaction                        10614
Event.HarborReport                        2487
Name: count, dtype: int64


The (directed) edges in the graph represent **events or relationships** between entities, either:
1. **VESSEL MOVEMENTS** – a vessel visiting a location (`Event.TransportEvent.TransponderPing`)
2. **HARBOR IMPORT RECORDS** – a commodity being delivered (`Event.Transaction`)
3. **HARBOR REPORTS** – or a harbor report being filed (`Event.HarborReport`)

In [33]:
# Display a few edges of each type
edge_data.groupby('type').head(1)

,source,target,key,type,time,dwell,date,data_author,aphorism,holiday_greeting,wisdom,saying of the sea
0,City of Haacklee,perchplundererbc0,0,Event.TransportEvent.TransponderPing,2035-09-16T04:06:48.185987,115074.790577,NaN,NaN,NaN,NaN,NaN,NaN
22674,cargo_2035_2394778c,gadusnspecificatae4ba,0,Event.Transaction,NaN,NaN,2035-11-03,NaN,NaN,NaN,NaN,NaN
33288,wavewranglerc2d,City of Haacklee,0,Event.HarborReport,NaN,NaN,2035-09-14,Portmaster of Haacklee,"The sea-shore is a sort of neutral ground, a m...",What are you doing for Saw Appreciation Day th...,"Boats, like whiskey, are all good.",An island is a world apart.


In [34]:
transponder_pings = edge_data[edge_data['type'] == 'Event.TransportEvent.TransponderPing']
transactions = edge_data[edge_data['type'] == 'Event.Transaction']
harbor_reports = edge_data[edge_data['type'] == 'Event.HarborReport']

In [35]:
print('TRANSPONDER PINGS:')
for col in transponder_pings.columns:
    missing = transponder_pings[col].isnull().sum()
    print(f"Missing values in '{col}': {missing} out of {len(transponder_pings)} total points")

print('\nTRANSACTIONS:')
for col in transactions.columns:
    missing = transactions[col].isnull().sum()
    print(f"Missing values in '{col}': {missing} out of {len(transactions)} total points")

print('\nHARBOR REPORTS:')
for col in harbor_reports.columns:
    missing = harbor_reports[col].isnull().sum()
    print(f"Missing values in '{col}': {missing} out of {len(harbor_reports)} total points")

TRANSPONDER PINGS:
Missing values in 'source': 0 out of 258542 total points
Missing values in 'target': 0 out of 258542 total points
Missing values in 'key': 0 out of 258542 total points
Missing values in 'type': 0 out of 258542 total points
Missing values in 'time': 0 out of 258542 total points
Missing values in 'dwell': 0 out of 258542 total points
Missing values in 'date': 258542 out of 258542 total points
Missing values in 'data_author': 258542 out of 258542 total points
Missing values in 'aphorism': 258542 out of 258542 total points
Missing values in 'holiday_greeting': 258542 out of 258542 total points
Missing values in 'wisdom': 258542 out of 258542 total points
Missing values in 'saying of the sea': 258542 out of 258542 total points

TRANSACTIONS:
Missing values in 'source': 0 out of 10614 total points
Missing values in 'target': 0 out of 10614 total points
Missing values in 'key': 0 out of 10614 total points
Missing values in 'type': 0 out of 10614 total points
Missing values 

We drop useless metadata from each edge type:

In [36]:
transponder_pings = drop_na_cols(transponder_pings)
transactions = drop_na_cols(transactions)
harbor_reports = drop_na_cols(harbor_reports)

Dropped columns (all NaN): ['date', 'data_author', 'aphorism', 'holiday_greeting', 'wisdom', 'saying of the sea']
Dropped columns (all NaN): ['time', 'dwell', 'data_author', 'aphorism', 'holiday_greeting', 'wisdom', 'saying of the sea']
Dropped columns (all NaN): ['time', 'dwell']


In [37]:
harbor_reports = harbor_reports.drop(columns=['aphorism', 'holiday_greeting', 'wisdom', 'saying of the sea'])

In [38]:
# Drop 'key' column from all dfs if it exists since it doesn't add useful information
transponder_pings = transponder_pings.drop(columns=['key'], errors='ignore')
harbor_reports = harbor_reports.drop(columns=['key'], errors='ignore')
transactions = transactions.drop(columns=['key'], errors='ignore')

Now we convert dwell and time to a readable and standard format:

In [39]:
transponder_pings['time']

0         2035-09-16T04:06:48.185987
1         2035-09-20T05:21:33.678120
2         2035-09-28T04:31:47.118191
3         2035-10-04T04:59:36.052683
4         2035-10-15T04:26:14.647650
                     ...            
271638    2035-09-20T23:27:08.743682
271639           2035-10-18T00:30:00
271640    2035-08-24T11:43:28.130654
271641           2035-08-29T14:30:00
271642    2035-11-12T02:52:42.477318
Name: time, Length: 258542, dtype: object

In [40]:
import re

# Add .000000 if the string does not already have microseconds
def add_microseconds(s): 
    return re.sub(r'(\d{2}:\d{2}:\d{2})(?!\.\d+)', r'\1.000000', s)

transponder_pings['time'] = transponder_pings['time'].astype(str).str.strip().apply(add_microseconds)

# Print
transponder_pings['time']

0         2035-09-16T04:06:48.185987
1         2035-09-20T05:21:33.678120
2         2035-09-28T04:31:47.118191
3         2035-10-04T04:59:36.052683
4         2035-10-15T04:26:14.647650
                     ...            
271638    2035-09-20T23:27:08.743682
271639    2035-10-18T00:30:00.000000
271640    2035-08-24T11:43:28.130654
271641    2035-08-29T14:30:00.000000
271642    2035-11-12T02:52:42.477318
Name: time, Length: 258542, dtype: object

In [41]:
# Convert 'time' column to datetime
transponder_pings['time'] = pd.to_datetime(transponder_pings['time'], errors='coerce')
transactions['date'] = pd.to_datetime(transactions['date'], errors='coerce')
harbor_reports['date'] = pd.to_datetime(harbor_reports['date'], errors='coerce')

# Adding a new column converting dwell from seconds to hours
transponder_pings['dwell_hours'] = transponder_pings['dwell'] / 3600

# Adding a new column converting dwell from hours to days
transponder_pings['dwell_days'] = transponder_pings['dwell_hours'] / 24

In [42]:
# Print count of null values in time columns

print(f"Null values in 'time' column of transponder_pings: {transponder_pings['time'].isnull().sum()}")
print(f"Null values in 'date' column of transactions: {transactions['date'].isnull().sum()}")
print(f"Null values in 'date' column of harbor_reports: {harbor_reports['date'].isnull().sum()}")

Null values in 'time' column of transponder_pings: 0
Null values in 'date' column of transactions: 0
Null values in 'date' column of harbor_reports: 0


In [43]:
for col in transponder_pings.columns:
    print(col, transponder_pings[col].dtype)

source object
target object
type object
time datetime64[ns]
dwell float64
dwell_hours float64
dwell_days float64


In [44]:
transponder_pings.head()

,source,target,type,time,dwell,dwell_hours,dwell_days
0,City of Haacklee,perchplundererbc0,Event.TransportEvent.TransponderPing,2035-09-16 04:06:48.185987,115074.790577,31.965220,1.331884
1,City of Haacklee,perchplundererbc0,Event.TransportEvent.TransponderPing,2035-09-20 05:21:33.678120,412706.321880,114.640645,4.776694
2,City of Haacklee,perchplundererbc0,Event.TransportEvent.TransponderPing,2035-09-28 04:31:47.118191,286092.881809,79.470245,3.311260
3,City of Haacklee,perchplundererbc0,Event.TransportEvent.TransponderPing,2035-10-04 04:59:36.052683,327623.947317,91.006652,3.791944
4,City of Haacklee,perchplundererbc0,Event.TransportEvent.TransponderPing,2035-10-15 04:26:14.647650,243225.352350,67.562598,2.815108


In [45]:
for col in transactions.columns:
    print(col, transactions[col].dtype)

source object
target object
type object
date datetime64[ns]


In [46]:
transactions.head()

,source,target,type,date
22674,cargo_2035_2394778c,gadusnspecificatae4ba,Event.Transaction,2035-11-03
22675,cargo_2035_2394778c,City of South Paackland,Event.Transaction,2035-11-03
22676,cargo_2035_23956ba0,gadusnspecificatae4ba,Event.Transaction,2035-08-16
22677,cargo_2035_23956ba0,City of South Paackland,Event.Transaction,2035-08-16
22678,cargo_2035_23957cfd,gadusnspecificatae4ba,Event.Transaction,2035-08-20


In [47]:
for col in harbor_reports.columns:
    print(col, harbor_reports[col].dtype)

source object
target object
type object
date datetime64[ns]
data_author object


In [48]:
harbor_reports.head()

,source,target,type,date,data_author
33288,wavewranglerc2d,City of Haacklee,Event.HarborReport,2035-09-14,Portmaster of Haacklee
33289,wavewranglerc2d,City of Haacklee,Event.HarborReport,2035-08-20,Portmaster of Haacklee
33290,wavewranglerc2d,City of Haacklee,Event.HarborReport,2035-09-17,Portmaster of Haacklee
33291,wavewranglerc2d,City of Himark,Event.HarborReport,2035-08-16,Portmaster of Himark
33292,wavewranglerc2d,City of Himark,Event.HarborReport,2035-09-21,Portmaster of Himark


### *Overview Table* of edge types

In [49]:
def get_node_type(node_id):
    # Try to get node type from node_data, fallback to index name
    if node_id in node_data.index:
        return node_data.loc[node_id]['type']
    return 'Unknown'

def get_other_attributes(row, skip_cols):
    return ', '.join([f"{col}={row[col]}" for col in row.index if col not in skip_cols and pd.notnull(row[col])])

overview = []
for df, label in zip(
    [transponder_pings, transactions, harbor_reports],
    ['Transponder Ping', 'Transaction', 'Harbor Report']
):
    for _, row in df.iterrows():
        event_type = row['type'] if 'type' in row else label
        source_type = get_node_type(row['source']) if 'source' in row else ''
        target_type = get_node_type(row['target']) if 'target' in row else ''
        other_attrs = get_other_attributes(row, ['type', 'source', 'target', 'key'])
        overview.append({
            'event type': event_type,
            'source type': source_type,
            'target type': target_type,
            'other attributes': other_attrs
        })

overview_df = pd.DataFrame(overview)
summary = (
    overview_df.groupby(['event type', 'source type', 'target type'])['other attributes']
    .agg(['count', lambda x: '; '.join(list(x)[:1])])  # show one example of other attributes
    .reset_index()
    .rename(columns={'<lambda_0>': 'example other attributes'})
)
print(summary.to_string(index=False))

                          event type                    source type                   target type  count                                                                                            example other attributes
                  Event.HarborReport      Entity.Vessel.CargoVessel          Entity.Location.City    914                                                        date=2035-08-24 00:00:00, data_author=Portmaster of Haacklee
                  Event.HarborReport      Entity.Vessel.Ferry.Cargo          Entity.Location.City     19                                                          date=2035-07-03 00:00:00, data_author=Portmaster of Himark
                  Event.HarborReport  Entity.Vessel.Ferry.Passenger          Entity.Location.City     34                                                        date=2035-10-18 00:00:00, data_author=Portmaster of Haacklee
                  Event.HarborReport    Entity.Vessel.FishingVessel          Entity.Location.City   1438            

### Export Cleaned Data to JSON

For easier handling and downstream analysis, the cleaned knowledge graph is exported as JSON files. These files preserve the entity and edge structure, making it straightforward to reload or share the data for further processing.

In [50]:
# Combine cleaned DataFrames for Vessels
vessels_cleaned = pd.concat([
    fishing_vessels, ferry_vessels, cargo_vessels,
    ferry_cargo_vessels, tour_vessels, research_vessels, other_vessels
])

# NOTE: vessels['style'].unique() > results in 1 unique value with NaN so this column is not useful
vessels_cleaned = vessels_cleaned.drop(columns=['style'], errors='ignore')

# Combine cleaned DataFrames for Locations
locations_cleaned = pd.concat([points, cities, regions])

# NOTE: Switch name and index in locations_cleaned for better readability
locations_cleaned = locations_cleaned.reset_index().rename(columns={'index': 'name', 'Name': 'id'}).set_index('id')

Final preprocessing to keep indexes as ID...

In [51]:
# Turn all column names to lowercase
vessels_cleaned.columns = vessels_cleaned.columns.str.lower()
locations_cleaned.columns = locations_cleaned.columns.str.lower()
documents.columns = documents.columns.str.lower()
commodities.columns = commodities.columns.str.lower()
transponder_pings.columns = transponder_pings.columns.str.lower()
transactions.columns = transactions.columns.str.lower()
harbor_reports.columns = harbor_reports.columns.str.lower()

# Reset index and rename it to 'id' before exporting to JSON
vessels_cleaned = vessels_cleaned.reset_index().rename(columns={'index': 'id'})
locations_cleaned = locations_cleaned.reset_index().rename(columns={'index': 'id'})
documents = documents.reset_index().rename(columns={'index': 'id'})
commodities = commodities.reset_index().rename(columns={'index': 'id'})

transponder_pings = transponder_pings.reset_index(drop=True).rename(columns={'index': 'id'})
transactions = transactions.reset_index(drop=True).rename(columns={'index': 'id'})
harbor_reports = harbor_reports.reset_index(drop=True).rename(columns={'index': 'id'})

Change location reference in events

In [52]:
# Create mapping from location name to id
location_name_to_id = dict(zip(locations_cleaned['name'], locations_cleaned['id']))

# Substitute harbor_reports['target'] with corresponding id
harbor_reports['target'] = harbor_reports['target'].map(location_name_to_id).fillna(harbor_reports['target'])

# Substitute transactions['target'] with corresponding id
transactions['target'] = transactions['target'].map(location_name_to_id).fillna(transactions['target'])

# Substitute transponder_pings['source'] with corresponding id
transponder_pings['source'] = transponder_pings['source'].map(location_name_to_id).fillna(transponder_pings['source'])

**NOTE**: To simplify graph structure and avoid redundancies, we can intervene on 'transactions' and 'documents'.

Merge documents and transactions. Update documents, adding 'commodity' column. The value should be found in transactions['target'] where it is a fish id.

Now keep only transactions where 'target' is a location.

In [53]:
documents.head()

,id,type,qty_tons,date
0,cargo_2035_2394778c,Entity.Document.DeliveryReport,24.375,2035-11-03
1,cargo_2035_23956ba0,Entity.Document.DeliveryReport,18.125,2035-08-16
2,cargo_2035_23957cfd,Entity.Document.DeliveryReport,20.625,2035-08-20
3,cargo_2035_23958501,Entity.Document.DeliveryReport,13.125,2035-11-07
4,cargo_2035_23959ab6,Entity.Document.DeliveryReport,13.125,2035-08-24


In [54]:
transactions.head()

,source,target,type,date
0,cargo_2035_2394778c,gadusnspecificatae4ba,Event.Transaction,2035-11-03
1,cargo_2035_2394778c,South Paackland,Event.Transaction,2035-11-03
2,cargo_2035_23956ba0,gadusnspecificatae4ba,Event.Transaction,2035-08-16
3,cargo_2035_23956ba0,South Paackland,Event.Transaction,2035-08-16
4,cargo_2035_23957cfd,gadusnspecificatae4ba,Event.Transaction,2035-08-20


In [55]:
# Create a mapping from transaction id to fish id (where target is a fish)
fish_mask = transactions['target'].isin(commodities['id'])

# Keep only rows where target is a fish and create mapping
transaction_fish = transactions[fish_mask]
transaction_fish_map = dict(zip(transaction_fish['source'], transaction_fish['target']))

# Add 'commodity' column to documents using the mapping
documents['commodity'] = documents['id'].map(transaction_fish_map)

# Drop 'date' since it's redundant with transactions['date']
documents = documents.drop(columns=['date'], errors='ignore')

# Keep only transactions where target is a location
location_mask = transactions['target'].isin(locations_cleaned['id'])
transactions = transactions[location_mask]

Drop 'type' from all datasets and preserve info only when useful (e.g. vessels, locations)

In [56]:
# Drop 'type' from all datasets and preserve info only when useful (e.g. vessels, locations)
transactions = transactions.drop(columns=['type'], errors='ignore')
transponder_pings = transponder_pings.drop(columns=['type'], errors='ignore')
harbor_reports = harbor_reports.drop(columns=['type'], errors='ignore')

commodities = commodities.drop(columns=['type'], errors='ignore')
documents = documents.drop(columns=['type'], errors='ignore')

# Transform 'type' into 'vessel_type' for vessels and 'location_type' for locations
vessels_cleaned = vessels_cleaned.rename(columns={'type': 'vessel_type'})
locations_cleaned = locations_cleaned.rename(columns={'type': 'location_type'})

# For better readability, shorten values in 'vessel_type' and 'location_type'
vessels_cleaned['vessel_type'] = vessels_cleaned['vessel_type'].str.replace('Entity.Vessel.', '', regex=False)
locations_cleaned['location_type'] = locations_cleaned['location_type'].str.replace('Entity.Location.', '', regex=False)

Drop 'dwell_hours' and 'dwell_days' to ensure smaller size of the file

In [57]:
transponder_pings = transponder_pings.drop(columns=['dwell_hours', 'dwell_days'], errors='ignore')

Check for double entries...

In [58]:
# Check double entries in harbor_reports, transactions and transponder_pings
duplicates_harbor_reports = harbor_reports[harbor_reports.duplicated(subset=['source', 'target', 'date'], keep=False)]
duplicates_transactions = transactions[transactions.duplicated(subset=['source', 'target', 'date'], keep=False)]
duplicates_transponder_pings = transponder_pings[transponder_pings.duplicated(subset=['source', 'target', 'time'], keep=False)] 

print(f"Found {len(duplicates_harbor_reports)} duplicate entries in harbor_reports based on 'source', 'target', and 'date'.")
print(f"Found {len(duplicates_transactions)} duplicate entries in transactions based on 'source', 'target', and 'date'.")
print(f"Found {len(duplicates_transponder_pings)} duplicate entries in transponder_pings based on 'source', 'target', and 'time'.")

# Delete duplicates in harbor_reports, transactions and transponder_pings
if not duplicates_harbor_reports.empty:
    harbor_reports = harbor_reports.drop_duplicates(subset=['source', 'target', 'date'], keep='first')
    print(f"After removing duplicates, harbor_reports now has {len(harbor_reports)} entries.")

if not duplicates_transactions.empty:
    transactions = transactions.drop_duplicates(subset=['source', 'target', 'date'], keep='first')
    print(f"After removing duplicates, transactions now has {len(transactions)} entries.")

if not duplicates_transponder_pings.empty:
    transponder_pings = transponder_pings.drop_duplicates(subset=['source', 'target', 'time'], keep='first')
    print(f"After removing duplicates, transponder_pings now has {len(transponder_pings)} entries.")

Found 1347 duplicate entries in harbor_reports based on 'source', 'target', and 'date'.
Found 0 duplicate entries in transactions based on 'source', 'target', and 'date'.
Found 4 duplicate entries in transponder_pings based on 'source', 'target', and 'time'.
After removing duplicates, harbor_reports now has 1697 entries.
After removing duplicates, transponder_pings now has 258540 entries.


Instead of having fish species name in locations_cleaned['fish_species_present'] (list of names separated by commas), reference short index in commodities. To do it, we replace value with commodities['id'] where commodities['name'] matches fish species name.

In [59]:
# Create mapping from fish species name to commodity id
fish_name_to_id = dict(zip(commodities['name'], commodities['id']))

def map_fish_species(species_list):
    if species_list is None:
        return []
    # Handle both string and list cases
    if isinstance(species_list, str):
        # Assume comma-separated string
        species = [s.strip() for s in species_list.split(',')]
    elif isinstance(species_list, list):
        species = species_list
    else:
        return []
    # Map each species name to its commodity id, only include if found
    return [fish_name_to_id[s] for s in species if s in fish_name_to_id]

locations_cleaned['fish_species_present'] = locations_cleaned['fish_species_present'].apply(map_fish_species)

In [62]:
# Check if documents has negative quantities for 'qty_tons'
negative_qty_documents = documents[documents['qty_tons'] < 0]
print(f"Found {len(negative_qty_documents)} documents with negative 'qty_tons' out of {len(documents)} total documents.")

if not negative_qty_documents.empty:
    print(negative_qty_documents)
    
    # Remove these entries
    documents = documents[documents['qty_tons'] >= 0]
    print(f"After removing negative quantities, documents now has {len(documents)} entries.")

Found 251 documents with negative 'qty_tons' out of 5307 total documents.
                       id qty_tons              commodity
36    cargo_2035_23992af5     -3.5       piscissapidum9b7
38    cargo_2035_23994469   -4.375       piscissapidum9b7
53    cargo_2035_24018829    -1.25  gadusnspecificatae4ba
61    cargo_2035_240329d3    -1.25  gadusnspecificatae4ba
137   cargo_2035_24146b8e     -6.5        piscesfrigus900
...                   ...      ...                    ...
5265  cargo_2035_31142cb2    -2.75  gadusnspecificatae4ba
5272  cargo_2035_31156e4e     -0.5  gadusnspecificatae4ba
5289  cargo_2035_311757d0     -1.0        habeaspisces4eb
5301  cargo_2035_31189bb6     -2.0        habeaspisces4eb
5304  cargo_2035_3119237c     -1.5        habeaspisces4eb

[251 rows x 3 columns]
After removing negative quantities, documents now has 5056 entries.


Convert cleaned CSV datasets to JSON and save

In [63]:
# Save Entities
vessels_cleaned.to_json("../data/vessels.json", orient='records', date_format='iso', indent=2)
locations_cleaned.to_json("../data/locations.json", orient='records', date_format='iso', indent=2)
documents.to_json("../data/documents.json", orient='records', date_format='iso', indent=2)
commodities.to_json("../data/commodities.json", orient='records', date_format='iso', indent=2)

# Save Events
transponder_pings.to_json("../data/transponder_pings.json", orient='records', date_format='iso', indent=2)
transactions.to_json("../data/transactions.json", orient='records', date_format='iso', indent=2)
harbor_reports.to_json("../data/harbor_reports.json", orient='records', date_format='iso', indent=2)

print("Cleaned Data exported to JSON.")

Cleaned Data exported to JSON.
